In [8]:
from elasticsearch import Elasticsearch

In [10]:
import json

class BaseObject():
    def to_json(self):
        return json.dumps(self.__dict__)

class Image(BaseObject):
    def __init__(self, user, _id, url, vector=[], labels=[]):
        self._id=_id
        self.url=url
        self.vector=vector
        self.labels=labels
        self.user=user

    def __str__(self):
        return f'{self.user}-{self.id}'

class Album(BaseObject):
    def __init__(self, user, _id, name, images=[]):
        self._id=_id
        self.name=name
        self.images=images
        self.user=user

    def __str__(self):
        return f'{self.user}-{self.name}'

In [14]:
import base64
import struct
def encode_vector(vector):
    data = bytearray()
    for v in vector:
        data.extend(bytearray(struct.pack('!f', v)))
    return base64.b64encode(data).decode('utf-8')

In [46]:
import random

In [82]:
class ESClient:
    def __init__(self):
        es_server = '35.184.175.102'
        es_user = 'elastic'
        es_pwd = 'espwd'
        self.es = Elasticsearch("http://{}:9200".format(es_server), http_auth=(es_user, es_pwd))
    
    def encode_vector(self, vector):
        data = bytearray()
        for v in vector:
            data.extend(bytearray(struct.pack('!f', v)))
        return base64.b64encode(data).decode('utf-8')
    
    def addImage(self, image):
        _id = image._id
        doc = {
            'id': _id,
            'user_id': image.user,
            'source': image.url
        }
        if image.labels:
            doc['labels'] = image.labels
        if image.vector:
            doc['vector'] = list(image.vector)
            doc['encoded_vector'] = self.encode_vector(image.vector)
        return self.es.index(index='images-index', doc_type='doc', id=_id, body=doc)
    
    def getImage(self, _id):
        d = self.es.get(index="images-index", doc_type='doc', id=_id).get('_source')
        im = Image(d.get('user_id'), _id, d.get('source'), vector=d.get('vector'), labels=d.get('labels'))
        return im
    
    def getAlbum(self, user, label, pageSize=20, page=0):
        q = {
          "size": pageSize,
          "from": page * pageSize,
          "query": {
            "bool": {
              "must": [
                {
                  "term": {
                    "user_id": {
                      "value": user
                    }
                  }
                },
                {
                  "term": {
                    "labels": label
                  }
                }
              ]
            }
          }
        }
        return self.es.search(index='images-index', body=q)
        
    
    def vectorSimilaritySearch(self, _id, pageSize=20, page=0):
        im = self.getImage(_id)
        vector = im.vector
        user = im.user
        
        q = {
          "size": pageSize,
          "from": page * pageSize,
          "_source": ["id", "user_id", "source", "labels"],
          "query": {
            "function_score": {
              "functions": [
                  {
                  "script_score": {
                      "script": {
                          "params": {
                              "seed_vector_string" : self.encode_vector(vector)
                          },
                          "source": "byte[] seed_bytes = Base64.getDecoder().decode(params.seed_vector_string); byte[] bytes = Base64.getDecoder().decode(doc['encoded_vector'].value); float[] seed_floats = new float[2048]; float[] floats = new float[2048]; for (int i = 0; i < 2048; i++) { int result = 0; int seed_result = 0; for (int j = 0; j < 4; j++) {result <<= 8; seed_result <<= 8; result |= (bytes[i*4+j] & 255); seed_result |= (seed_bytes[i*4+j] & 255); } seed_floats[i] = Float.intBitsToFloat(seed_result);  floats[i] = Float.intBitsToFloat(result); } float top = 0, bottom_seed = 0, bottom = 0; for(int i = 0; i < 2048; i++) {top += floats[i] * seed_floats[i]; bottom_seed += seed_floats[i]* seed_floats[i]; bottom += floats[i]*floats[i]; } return top/(Math.sqrt(bottom)*Math.sqrt(bottom_seed))"
                      }
                  }
                  }
              ],
              "score_mode": "sum",
              "boost_mode": "sum",
              "query": {
                "bool": {
                    "must": [
                        {
                          "exists": {
                            "field": "encoded_vector"
                          }
                        },
                        {
                            "term": {
                                "user_id": {
                                    "value": user
                                } 
                            }
                        }
                    ]
                }
              }
            }
          }
        }
        return [hit.get('_source') for hit in self.es.search(index='images-index', body=q).get('hits', {}).get('hits', [])]
        
#     def addAlbum():
    
#     def searchAlbum

In [83]:
es = ESClient()

In [79]:
# im = Image('markhuds', '1234567890', 'hack-sc-project/images/markhuds/party.jpeg', 
#            labels=['mark', 'is', 'really', 'cool'], 
#            vector=[random.random() for i in range(2048)])

In [84]:
# es.getImage('1').__dict__

In [85]:
es.vectorSimilaritySearch('1')

[{'user_id': 'markhuds', 'id': '1', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '415', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '875', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '782', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '532', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '542', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '519', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '118', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '40', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '295', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '677', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '758', 'source': '', 'labels': ['testing']},
 {'user_id': 'markhuds', 'id': '84', 'source': '', 'labels': ['testing']},
 {'user_id': 'ma

In [55]:
for i in range(1000):
    print(i)
    im = Image('markhuds', str(i), '', labels=['testing'], vector=[random.random() for i in range(2048)])
    es.addImage(im)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [56]:
es.es.get(index="images-index", doc_type='doc', id='1')

{'_index': 'images-index',
 '_type': 'doc',
 '_id': '1',
 '_version': 1,
 'found': True,
 '_source': {'id': '1',
  'user_id': 'markhuds',
  'source': '',
  'labels': ['testing'],
  'vector': [0.5524070419020654,
   0.10737648989168846,
   0.31461736189308054,
   0.05873369256051686,
   0.5160735381402252,
   0.42250151179080264,
   0.9279370292287371,
   0.9421736050682392,
   0.23506152447447481,
   0.7228073708894542,
   0.9614647434339513,
   0.8514296836907255,
   0.11128460116513694,
   0.6894384913534455,
   0.0660285461907898,
   0.5762857569070795,
   0.6038300605260636,
   0.7992757260544436,
   0.19663934198232258,
   0.989513352051589,
   0.7028814158597271,
   0.5533097123417128,
   0.8952510177693548,
   0.9274804206345326,
   0.9271187060155904,
   0.15566461034232393,
   0.6071894793532804,
   0.5916964436621794,
   0.36010621429117873,
   0.029150841275187078,
   0.4251671561268139,
   0.2511381668920868,
   0.7151385118931618,
   0.898133433874515,
   0.113993667224929